# **Perspectiva General de Mercado - DatAlpine**
*Filtro de tipo de mercado*

In [3]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
import numpy as np

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()
# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_")  and "puebla" in nombre and 'q' not in nombre
    and 'financiamientos' not in nombre
    and 'poblacion' not in nombre
    and 'salarios' not in nombre
    and 'publicacion' not in nombre
    
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_agosto_2024_puebla',
 'df_alfa_febrero_2024_puebla',
 'df_alfa_julio_2024_puebla',
 'df_alfa_junio_2024_puebla',
 'df_alfa_septiembre_2024_puebla',
 'df_jul_2023_puebla',
 'df_mar_2024_puebla',
 'df_may_2024_puebla',
 'df_sep_2023_puebla']

In [16]:
df_jul_2023_puebla.columns

Index(['id_jul_23', 'Q', 'categoria', 'ids', 'propiedad', 'precio',
       'metros_total', 'metros_construido', 'Precio_M2_Terreno',
       'precio_m2_construido', 'Personas_interesadas', 'Tiempo_publicacion',
       'Promedio_Interes_Mensual', 'Estado', 'Tipo', 'estacionamiento',
       'recamaras', 'baño', 'medio_baño', 'baño_total', 'Cantidad_de_pisos',
       'Antiguedad', 'Seguridad', 'Fraccionamiento', 'Colonia', 'CP', 'url'],
      dtype='object')

In [9]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    df.rename(columns={'Categoria':'categoria','Category':'categoria','Propiedad':'propiedad','Precio':'precio','Metros_total':'metros_total','m_total':'metros_total','m_construido':'metros_construido','Metros_construido':'metros_construido','Estacionamientos':'estacionamiento','Recamaras':'recamaras','Banos':'baño','Medio_banos':'medio_baño','Banos_Total':'baño_total','Precio_M2_Construido':'precio_m2_construido'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 
print("Columnas renombradas en los DataFrames filtrados.")

Columnas renombradas en los DataFrames filtrados.


In [57]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['categoria','propiedad','precio','metros_total','metros_construido', 'precio_m2_construido','estacionamiento','recamaras', 'baño', 'medio_baño', 'baño_total']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

In [58]:
df_total = pd.concat(dataframes_list, ignore_index=True)

In [59]:
df_total.columns

Index(['categoria', 'propiedad', 'precio', 'metros_total', 'metros_construido',
       'precio_m2_construido', 'estacionamiento', 'recamaras', 'baño',
       'medio_baño', 'baño_total'],
      dtype='object')

In [60]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32340 entries, 0 to 32339
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   categoria             32340 non-null  object 
 1   propiedad             32340 non-null  object 
 2   precio                32340 non-null  float64
 3   metros_total          32340 non-null  int64  
 4   metros_construido     32340 non-null  int64  
 5   precio_m2_construido  32340 non-null  float64
 6   estacionamiento       32340 non-null  int64  
 7   recamaras             32340 non-null  int64  
 8   baño                  32340 non-null  int64  
 9   medio_baño            32340 non-null  float64
 10  baño_total            32340 non-null  float64
dtypes: float64(4), int64(5), object(2)
memory usage: 2.7+ MB


In [61]:
rangos_precio = {
    "E1": (0, 500000),
    "E2": (500000, 750000),
    "E3": (750001, 1000000),
    "D1": (1000001, 1250000),
    "D2": (1250001, 1500000),
    "D3": (1500001, 1750000),
    "C1": (1750001, 2000000),
    "C2": (2000001, 2250000),
    "C3": (2250001, 2500000),
    "B1": (2500001, 2750000),  
    "B2": (2750001, 3000000),  
    "B3": (3000001, 3250000),  
    "A1": (3250001, 3500000),
    "A2": (3500001, 3750000),
    "A3": (3750001, 4000000),
    "S1": (4000001, 6000000),
    "S2": (6000001, 8000000),
    "S3": (8000001, 12000000),
    "L1": (12000001, 14000000),
    "L2": (14000001, 16000000),
    "L3": (16000001, 18000000),
    "L+": (18000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Función para asignar la categoría según el precio
def asignar_categoria(precio):
    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
        if limite_inferior <= precio < limite_superior:
            return categoria
    return None  # En caso de que el precio no caiga en ningún rango (caso raro)

df_total = df_total.drop(columns=['categoria'])
df_total['categoria'] = df_total['precio'].apply(asignar_categoria)

In [62]:
import numpy as np 
condition = (df_total['baño_total'] % 1 == 0.5) # Valores que contienen .5    
# Convertir al número entero anterior
df_total.loc[condition, 'baño_total'] = np.floor(df_total['baño_total']) 
df_total['baño_total'] = df_total['baño_total'].clip(lower=1,upper=8) # Limitar los valores 
df_total['estacionamiento'] = df_total['estacionamiento'].clip(lower=1,) #upper=8
df_total['recamaras'] = df_total['recamaras'].clip(lower=1,)#upper=8 

- Casa promedio
- Casa Mediana
- Precio mercado

- m2 Terreno
- M2 Construcción
- Precio m2 construcción

# Casa promedio

In [30]:
df_total['segmento'] = df_total['categoria'].str[0]
grupo_promedios = round(df_total.groupby('segmento')[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
grupo_promedios.astype(int)
# Renombrar las columnas para mayor claridad
grupo_promedios.rename(columns={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)

# Calcular el promedio de estacionamientos, recámaras y baños en general (sin segmento específico)
promedios_generales = round(df_total[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
promedios_generales = promedios_generales.astype(int)
# Renombrar las columnas para mayor claridad
promedios_generales.rename(index={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)
# Agregar los promedios generales al dataframe de grupo_promedios
promedios_generales = pd.DataFrame(promedios_generales).T
promedios_generales['segmento'] = 'general'

casa_promedio = pd.concat([grupo_promedios, promedios_generales.set_index('segmento')], ignore_index=False)



print(grupo_promedios)
print(promedios_generales)
print(casa_promedio)


# Convertir el dataframe a JSON
grupo_promedios_json = casa_promedio.to_json(orient='index')

# Guardar el JSON en un archivo
with open('assets/tablas/grupo_casa_promedio.json', 'w') as f:
    f.write(grupo_promedios_json)

          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        3.0    2.0
B                      2.0        3.0    2.0
C                      2.0        3.0    2.0
D                      2.0        3.0    2.0
E                      2.0        3.0    2.0
L                      4.0        4.0    3.0
S                      3.0        3.0    3.0
   Estacionamientos  Recámaras  Baños segmento
0                 2          3      2  general
          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        3.0    2.0
B                      2.0        3.0    2.0
C                      2.0        3.0    2.0
D                      2.0        3.0    2.0
E                      2.0        3.0    2.0
L                      4.0        4.0    3.0
S                      3.0        3.0    3.0
general                2.0        3.0    2.0


# Casa Mediana

In [31]:
df_total['segmento'] = df_total['categoria'].str[0]
grupo_mediana = round(df_total.groupby('segmento')[['estacionamiento', 'recamaras', 'baño_total']].median(),0)
grupo_mediana.astype(int)
# Renombrar las columnas para mayor claridad
grupo_mediana.rename(columns={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)
print(grupo_mediana)

# Calcular mediana de estacionamientos, recámaras y baños en general (sin segmento específico)
mediana_general = round(df_total[['estacionamiento', 'recamaras', 'baño_total']].median(),0)
mediana_general = mediana_general.astype(int)
# Renombrar las columnas para mayor claridad
mediana_general.rename(index={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)
mediana_general = pd.DataFrame(mediana_general).T
mediana_general['segmento'] = 'general'
print(mediana_general)

casa_media = pd.concat([grupo_mediana, mediana_general.set_index('segmento')], ignore_index=False)
print(casa_media)

# Guardar el JSON en un archivo
grupo_promedios_json = casa_media.to_json(orient='index')
with open('assets/tablas/grupo_casa_mediana.json', 'w') as f:
    f.write(grupo_promedios_json)

          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        3.0    2.0
B                      2.0        3.0    2.0
C                      2.0        3.0    2.0
D                      2.0        3.0    2.0
E                      2.0        3.0    2.0
L                      4.0        4.0    3.0
S                      2.0        3.0    3.0
   Estacionamientos  Recámaras  Baños segmento
0                 2          3      2  general
          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        3.0    2.0
B                      2.0        3.0    2.0
C                      2.0        3.0    2.0
D                      2.0        3.0    2.0
E                      2.0        3.0    2.0
L                      4.0        4.0    3.0
S                      2.0        3.0    3.0
general                2.0        3.0    2.0


# Precio Mercado

In [32]:
df_total = df_total[df_total['precio'] != 0]

In [33]:
grupo_descripcion = df_total.groupby('segmento')['precio'].describe().reset_index()
grupo_descripcion = grupo_descripcion[['segmento','min', 'max','mean','50%']] #Columnas deseadas
grupo_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)
grupo_descripcion 

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,3251529.0,3999000.0,3.634741e+06,3607000.0
1,B,2501000.0,3249000.0,2.866111e+06,2850000.0
2,C,1754500.0,2499000.0,2.126950e+06,2150000.0
3,D,1005000.0,1749000.0,1.384752e+06,1380000.0
4,E,148640.0,335410000.0,2.638442e+06,700000.0
5,L,12200000.0,21995000.0,1.548745e+07,15000000.0
6,S,4017116.0,11990000.0,6.265449e+06,5500000.0


In [34]:
# Función para formatear valores en formato monetario
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato con $ y 2 decimales, separador de miles

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 5):  # 1, 2, 3, 4 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    grupo_descripcion.iloc[:, i] = grupo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
grupo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_5576\913539489.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $3,251,529.00
1     $2,501,000.00
2     $1,754,500.00
3     $1,005,000.00
4       $148,640.00
5    $12,200,000.00
6     $4,017,116.00
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion.iloc[:, i] = grupo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_5576\913539489.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0      $3,999,000.00
1      $3,249,000.00
2      $2,499,000.00
3      $1,749,000.00
4    $335,410,000.00
5     $21,995,000.00
6     $11,990,000.00
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_de

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,"$3,251,529.00","$3,999,000.00","$3,634,740.61","$3,607,000.00"
1,B,"$2,501,000.00","$3,249,000.00","$2,866,111.15","$2,850,000.00"
2,C,"$1,754,500.00","$2,499,000.00","$2,126,949.61","$2,150,000.00"
3,D,"$1,005,000.00","$1,749,000.00","$1,384,751.98","$1,380,000.00"
4,E,"$148,640.00","$335,410,000.00","$2,638,441.56","$700,000.00"
5,L,"$12,200,000.00","$21,995,000.00","$15,487,454.99","$15,000,000.00"
6,S,"$4,017,116.00","$11,990,000.00","$6,265,448.61","$5,500,000.00"


In [35]:
todo_descripcion = df_total['precio'].describe()[['min', 'max', 'mean', '50%']].to_frame().T
todo_descripcion['segmento'] = 'general'

todo_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores
for i in range(4):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_5576\2840734770.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio    $148,640.00
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_5576\2840734770.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio    $335,410,000.00
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_5576\2840734770.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise

,Mínimo,Máximo,Promedio,Mediana,segmento
precio,"$148,640.00","$335,410,000.00","$3,888,710.77","$2,590,000.00",general


In [38]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([grupo_descripcion, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_precio_mercado.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [39]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,"$3,251,529.00","$3,999,000.00","$3,634,740.61","$3,607,000.00"
1,B,"$2,501,000.00","$3,249,000.00","$2,866,111.15","$2,850,000.00"
2,C,"$1,754,500.00","$2,499,000.00","$2,126,949.61","$2,150,000.00"
3,D,"$1,005,000.00","$1,749,000.00","$1,384,751.98","$1,380,000.00"
4,E,"$148,640.00","$335,410,000.00","$2,638,441.56","$700,000.00"
5,L,"$12,200,000.00","$21,995,000.00","$15,487,454.99","$15,000,000.00"
6,S,"$4,017,116.00","$11,990,000.00","$6,265,448.61","$5,500,000.00"
7,general,"$148,640.00","$335,410,000.00","$3,888,710.77","$2,590,000.00"


# M2 Terreno

In [40]:
df_total_terreno = df_total[df_total['metros_total'] != 0]
df_descripcion = df_total_terreno.groupby('segmento')['metros_total'].describe().reset_index()
df_descripcion = df_descripcion[['segmento', 'min', 'max', 'mean', '50%']] #Columnas deseadas
df_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)


# Función para formatear valores en formato metros
def formato_metros(valor):
    return f"{valor:,.2f} m²"

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 5):  # 1, 2, 3, 4 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
df_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_5576\3801508271.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    48.00 m²
1    43.00 m²
2    28.00 m²
3    33.00 m²
4    30.00 m²
5    80.00 m²
6    42.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_5576\3801508271.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       22,715.00 m²
1       26,964.00 m²
2       19,124.00 m²
3       12,289.00 m²
4      220,000.00 m²
5        9,288.00 m²
6    7,500,000.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].a

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,48.00 m²,"22,715.00 m²",250.94 m²,137.00 m²
1,B,43.00 m²,"26,964.00 m²",249.80 m²,125.00 m²
2,C,28.00 m²,"19,124.00 m²",252.94 m²,101.00 m²
3,D,33.00 m²,"12,289.00 m²",148.06 m²,93.00 m²
4,E,30.00 m²,"220,000.00 m²",332.66 m²,150.00 m²
5,L,80.00 m²,"9,288.00 m²",831.28 m²,682.00 m²
6,S,42.00 m²,"7,500,000.00 m²","2,874.34 m²",250.00 m²


In [41]:
todo_descripcion = df_total[df_total['metros_total'] != 0]['metros_total'].describe()[['min', 'max', 'mean', '50%']].to_frame().T
todo_descripcion['segmento'] = 'general'

todo_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores
for i in range(4):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_5576\3501913492.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    28.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_5576\3501913492.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    7,500,000.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_5576\3501913492.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will ra

,Mínimo,Máximo,Promedio,Mediana,segmento
metros_total,28.00 m²,"7,500,000.00 m²",811.86 m²,145.00 m²,general


In [42]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([df_descripcion, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_terreno.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [43]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,48.00 m²,"22,715.00 m²",250.94 m²,137.00 m²
1,B,43.00 m²,"26,964.00 m²",249.80 m²,125.00 m²
2,C,28.00 m²,"19,124.00 m²",252.94 m²,101.00 m²
3,D,33.00 m²,"12,289.00 m²",148.06 m²,93.00 m²
4,E,30.00 m²,"220,000.00 m²",332.66 m²,150.00 m²
5,L,80.00 m²,"9,288.00 m²",831.28 m²,682.00 m²
6,S,42.00 m²,"7,500,000.00 m²","2,874.34 m²",250.00 m²
7,general,28.00 m²,"7,500,000.00 m²",811.86 m²,145.00 m²


# M2 Construcción

In [44]:
df_total_terreno = df_total[~df_total['metros_construido'].isin([0, 1,2,3])]
df_descripcion = df_total_terreno.groupby('segmento')['metros_construido'].describe().reset_index()
df_descripcion = df_descripcion[['segmento', 'min', 'max', 'mean', '50%']] #Columnas deseadas
df_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)


# Función para formatear valores en formato metros
def formato_metros(valor):
    return f"{valor:,.2f} m²"

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 5):  # 1, 2, 3, 4 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
df_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_5576\2035657892.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    46.00 m²
1    55.00 m²
2    28.00 m²
3    33.00 m²
4    30.00 m²
5    78.00 m²
6    55.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_5576\2035657892.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       17,468.00 m²
1       14,583.00 m²
2       40,642.00 m²
3       12,289.00 m²
4      220,000.00 m²
5        9,288.00 m²
6    7,500,000.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].a

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,46.00 m²,"17,468.00 m²",266.97 m²,196.00 m²
1,B,55.00 m²,"14,583.00 m²",242.28 m²,149.00 m²
2,C,28.00 m²,"40,642.00 m²",277.33 m²,118.00 m²
3,D,33.00 m²,"12,289.00 m²",153.24 m²,100.00 m²
4,E,30.00 m²,"220,000.00 m²",320.34 m²,180.00 m²
5,L,78.00 m²,"9,288.00 m²",644.76 m²,566.00 m²
6,S,55.00 m²,"7,500,000.00 m²","2,830.29 m²",300.00 m²


In [45]:
todo_descripcion = df_total[~df_total['metros_construido'].isin([0, 1,2,3])]['metros_total'].describe()[['min', 'max', 'mean', '50%']].to_frame().T
todo_descripcion['segmento'] = 'general'

todo_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores
for i in range(4):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_5576\641405135.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    28.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_5576\641405135.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    7,500,000.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_5576\641405135.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise

,Mínimo,Máximo,Promedio,Mediana,segmento
metros_total,28.00 m²,"7,500,000.00 m²",811.86 m²,145.00 m²,general


In [46]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([df_descripcion, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_construidos.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [48]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,46.00 m²,"17,468.00 m²",266.97 m²,196.00 m²
1,B,55.00 m²,"14,583.00 m²",242.28 m²,149.00 m²
2,C,28.00 m²,"40,642.00 m²",277.33 m²,118.00 m²
3,D,33.00 m²,"12,289.00 m²",153.24 m²,100.00 m²
4,E,30.00 m²,"220,000.00 m²",320.34 m²,180.00 m²
5,L,78.00 m²,"9,288.00 m²",644.76 m²,566.00 m²
6,S,55.00 m²,"7,500,000.00 m²","2,830.29 m²",300.00 m²
7,general,28.00 m²,"7,500,000.00 m²",811.86 m²,145.00 m²


# Pecio M2 Construcción

In [51]:
df_total_terreno = df_total[~df_total['metros_construido'].isin([0, 1,2,3])]
df_total_terreno = df_total_terreno[df_total_terreno['precio_m2_construido'] > 10]

grupo_filtrado = df_total_terreno.groupby('segmento')['precio_m2_construido'].describe().reset_index()
# Mantener solo las columnas que deseas (en este caso, min y max)
grupo_filtrado = grupo_filtrado[['segmento', 'min', 'max', 'mean', '50%']] #Columnas deseadas
grupo_filtrado.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores, excluyendo la columna 'segmento'
for i in range(1, 5):  # 1, 2, 3, 4 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    grupo_filtrado.iloc[:, i] = grupo_filtrado.iloc[:, i].astype(float).apply(formato_monetario)

grupo_filtrado 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_5576\3813107853.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0      $213.98
1      $205.00
2       $44.29
3      $137.52
4      $128.06
5    $1,399.66
6      $146.03
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_filtrado.iloc[:, i] = grupo_filtrado.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_5576\3813107853.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $78,260.87
1     $54,102.27
2     $86,250.00
3     $45,945.95
4    $793,310.42
5    $189,285.71
6    $136,363.64
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_filtrado.iloc[:, i] = grupo_filtrado.iloc[:, i].astype(float).apply

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,$213.98,"$78,260.87","$22,393.05","$18,720.38"
1,B,$205.00,"$54,102.27","$19,933.43","$19,021.74"
2,C,$44.29,"$86,250.00","$18,167.46","$17,638.66"
3,D,$137.52,"$45,945.95","$13,577.63","$13,861.39"
4,E,$128.06,"$793,310.42","$8,282.45","$4,313.81"
5,L,"$1,399.66","$189,285.71","$28,936.56","$26,000.00"
6,S,$146.03,"$136,363.64","$22,117.87","$19,541.96"


In [53]:
df_total_terreno = df_total_terreno[df_total_terreno['precio_m2_construido'] > 10]
todo_descripcion = df_total_terreno[~df_total_terreno['metros_construido'].isin([0, 1,2,3])]['precio_m2_construido'].describe()[['min', 'max', 'mean', '50%']].to_frame().T
todo_descripcion['segmento'] = 'general'

todo_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores
for i in range(4):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_5576\3521683632.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $44.29
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_5576\3521683632.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $793,310.42
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_5576\3521683632.py:14: FutureWarning: Setting an item of incompatible dtype is deprec

,Mínimo,Máximo,Promedio,Mediana,segmento
precio_m2_construido,$44.29,"$793,310.42","$17,400.22","$16,500.00",general


In [54]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([grupo_filtrado, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_precio_construidos.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [55]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,$213.98,"$78,260.87","$22,393.05","$18,720.38"
1,B,$205.00,"$54,102.27","$19,933.43","$19,021.74"
2,C,$44.29,"$86,250.00","$18,167.46","$17,638.66"
3,D,$137.52,"$45,945.95","$13,577.63","$13,861.39"
4,E,$128.06,"$793,310.42","$8,282.45","$4,313.81"
5,L,"$1,399.66","$189,285.71","$28,936.56","$26,000.00"
6,S,$146.03,"$136,363.64","$22,117.87","$19,541.96"
7,general,$44.29,"$793,310.42","$17,400.22","$16,500.00"
